In [8]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import re
import os

In [9]:
dirpath = os.getcwd()
videopath = os.path.join(dirpath, "..", "..", "udder_video", "video_metadata")
video =  pd.read_csv(os.path.join(videopath, "video_metadata_20231117.csv"))
data_path = os.path.join(dirpath, "..", "data")

In [10]:
milk_df= pd.read_csv(os.path.join(data_path, "delpro_visit_lactation.csv")) # , header = 0,  names = names, dtype = dtypes)
milk_df = milk_df[milk_df['date'].notna()]
# date formats: date, begin time, end time
milk_df.date = [datetime.strptime(day, "%Y-%m-%d") for day in milk_df.date]
milk_df.begin_time = [datetime.strptime(day, "%m/%d/%Y %I:%M %p") for day in milk_df.begin_time]
# milk report time is in AM -PM format
milk_df["start_hour"]=[date.hour for date in milk_df.begin_time]
milk_df["day"]=[date.day for date in milk_df.begin_time]
milk_df["month"]=[date.month for date in milk_df.begin_time]
milk_df["year"]=[date.year for date in milk_df.begin_time]
milk_df = milk_df[(milk_df.day == 17) & (milk_df.month == 11) & (milk_df.year == 2023)].reset_index()
milk_df = milk_df.drop(["date","day", "year", "month"], axis = 1)

# fix time and date to make them match
# I give a +-10 min margin form video to milking
video["start_hour0"]=[datetime.strptime(date, '%H:%M:%S').hour for date in video.time]
video["start_hour1"]=[datetime.strptime(date, '%H:%M:%S') + timedelta(minutes = 10) for date in video.time]
video["start_hour1"]=[hour.hour for hour in video.start_hour1]
video["start_hour2"]=[datetime.strptime(date, '%H:%M:%S') - timedelta(minutes = 10) for date in video.time]
video["start_hour2"]=[hour.hour for hour in video.start_hour2]

milk_videos = pd.concat([pd.merge(video, milk_df,  left_on=["cow", "start_hour0"], right_on=["animal_number", "start_hour"]),  \
                         pd.merge(video, milk_df,  left_on=["cow", "start_hour1"], right_on=["animal_number", "start_hour"]), \
                         pd.merge(video, milk_df,  left_on=["cow", "start_hour2"], right_on=["animal_number", "start_hour"])],\
                        axis= 0, ignore_index = True).drop_duplicates().reset_index(drop = True)

milk_videos = milk_videos.drop(["start_hour", "start_hour0", "start_hour1", "start_hour2"], axis = 1)
milk_videos.to_csv(os.path.join(data_path, "milk_videos_visit.csv"), index = False)

In [11]:
milk_videos.columns

Index(['cow', 'farmname', 'robotside', 'filename', 'directory', 'date', 'time',
       'size', 'computer', 'index', 'animal_number', 'lactation_number',
       'begin_time', 'end_time', 'yield', 'days_in_milk', 'conductivity_lf',
       'conductivity_lr', 'conductivity_rf', 'conductivity_rr', 'mean_flow_lf',
       'mean_flow_lr', 'mean_flow_rf', 'mean_flow_rr', 'milk_flow_duration',
       'occ', 'peak_flow_lr', 'peak_flow_rf', 'peak_flow_rr', 'peak_flow_lf',
       'teats_not_found', 'yield_lf', 'yield_lr', 'yield_rf', 'yield_rr',
       'interval', 'kickoff', 'kickoff_lf', 'kickoff_lr', 'kickoff_milkings',
       'kickoff_rf', 'kickoff_rr', 'alternative_attach', 'device_name',
       'incomplete', 'is_incomplete', 'incomplete_milkings'],
      dtype='object')

In [12]:
lac_df= pd.read_csv(os.path.join(data_path, "merged2_lactation.csv"))
lac_videos = pd.merge(video, lac_df, how = "left",  left_on="cow", right_on="animal_number")
lac_videos = lac_videos.drop(["start_hour0", "start_hour1", "start_hour2"], axis = 1)

In [15]:
lac_videos.drop(["start_hour0", "start_hour1", "start_hour2"], axis = 1)

,cow,farmname,robotside,filename,directory,date,time,size,computer,animal_number,...,kickoff_rf_perc,kickoff_rr_perc,kickoff_front_perc,kickoff_back_perc,kickoff_any_perc,yield_lf_visit_sum,yield_lr_visit_sum,yield_rr_visit_sum,yield_rf_visit_sum,interval_visit_mean
0,764,laufenberg,right,0764_20231117_144922.bag,videos_1,20231117,14:49:54,3807979957,maria,764,...,0.409277,0.136426,1.091405,0.545703,0.954980,3760.88,4127.59,1047.17,3398.21,33631.328786
1,902,laufenberg,right,0902_20231117_142952.bag,videos_1,20231117,14:30:30,4303784100,maria,902,...,NaN,30.635838,5.202312,34.104046,34.104046,747.62,948.01,1109.72,NaN,32111.132948
2,970,laufenberg,right,0970_20231117_150812.bag,videos_1,20231117,15:08:38,2800097357,maria,970,...,0.000000,0.398406,0.498008,0.498008,0.498008,3641.17,4326.03,4769.17,3237.11,28816.133466
3,973,laufenberg,right,0973_20231117_135700.bag,videos_1,20231117,13:57:48,5620008570,maria,973,...,0.000000,1.482326,0.000000,2.052452,2.052452,4351.28,3788.53,3573.59,3981.48,32205.709236
4,1116,laufenberg,right,1116_20231117_141845.bag,videos_1,20231117,14:19:42,6708885177,maria,1116,...,0.000000,1.544402,0.386100,1.930502,1.930502,1057.34,1573.67,1419.56,1154.97,34678.200772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1282,laufenberg,left,1282_20231117_121652.bag,videos_3,20231117,12:17:14,2518137301,lab,1282,...,0.000000,NaN,0.000000,0.222222,0.222222,4442.61,0.00,NaN,4082.49,28766.321111
158,1319,laufenberg,left,1319_20231117_122525.bag,videos_3,20231117,12:25:54,3337742803,lab,1319,...,0.000000,2.666667,0.000000,5.333333,5.333333,100.74,394.51,293.58,5.90,34286.120000
159,921,laufenberg,left,921_20231117_104851.bag,videos_3,20231117,10:49:00,913513850,lab,921,...,0.136426,0.136426,0.818554,0.272851,0.409277,2707.60,3741.13,3533.78,2777.11,31191.443383
160,935,laufenberg,left,935_20231117_115800.bag,videos_3,20231117,11:58:34,3849093180,lab,935,...,0.000000,0.000000,0.000000,0.108108,0.108108,4937.87,5038.16,4610.28,15.07,31778.222703
